This is a FNO solver for Reactive Burgers Equation. We have to specify the parameters here only, and the train()
function will train the operator. Analyze() function will solve the equation with FNO uptil FinalSolve time. It will also generate the figures to visualize and give L2 and Inf error between the true and original

In [50]:
import timeit
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
import matplotlib.pyplot as plt

import operator
from functools import reduce
from functools import partial
from timeit import default_timer
from utilities3 import *

from Adam import Adam

%run ./Train.ipynb
%run ./Analyze.ipynb
%run ./Data_Generator/KFRDG.ipynb

In [58]:
################################################################
#  configurations for Data Generation
################################################################

Alpha = 4.85                 #the equation parameter
InitialSolve = 350                 #the time for solving the weno initially
Space = 5000                       #the area from where training data is randomly is generated
NdtList = [15]                     #the labels are Ndt after the input
TrainingSamples = 1500
FinalSolve = 10000                 #for comparison

################################################################
#  configurations for FNO
################################################################
ntrain = 1450
ntest = 100

#sub = 2**3 #subsampling rate
#h = 2**13 // sub #total grid size divided by the subsampling rate
S = 200
dx = S+7

batch_size = 30
learning_rate = 0.01

epochs = 500
step_size = 50
gamma = 0.5

modes = 17
width = 72

################################################################
#  configurations for Figures
################################################################
Initial = 350
Cutoff = 500

In [59]:
for Ndt in NdtList: 
    Param = {
        "Alpha": Alpha,
        "InitialSolve": InitialSolve,
        "FinalSolve": FinalSolve,
        "Ndt": Ndt,
        "TrainingSamples": TrainingSamples,
        "ntrain": ntrain,
        "ntest": ntest,
        "S": S,
        "batch_size": batch_size,
        "learning_rate": learning_rate,
        "epochs": epochs,
        "step_size": step_size,
        "gamma": gamma, 
        "modes": modes,
        "width": width,
        "Initial": Initial,
        "Cutoff": Cutoff,
        "Space": Space,
        "FiguresOnly": False
    }
    #Udict = DataGenerator_Alpha(Alpha, TrainingSamples, Ndt, Space, InitialSolve)
    path_x = 'Data_Generator/' + f'input_{Alpha}_{InitialSolve}_{Ndt}_{TrainingSamples}_{Space}_{S}.npy'
    if not os.path.exists(path_x):
        Udict = DataGenerator_Alpha(Alpha, TrainingSamples, Ndt, S, dx, Space, InitialSolve)
    #Train the FNO model
    Train(Param)
    #Check the FNO model and savefigures
    Inf, L2 = Analyze(Param)

amp_0alpha_4.85 =-=-=-=-=-= 23.49s
Completed
0 0.6738490180578083 0.5189231625895909 0.33619277275841813 0.046675453782081607
1 0.5922846950124949 0.001279419877837241 0.05056648862772974 0.05926860809326172
2 0.5783844559919089 0.0005447642306906494 0.032930449082933626 0.03108122706413269
3 0.5826265818905085 0.001992724245480661 0.04988342256381594 0.04672417998313904
4 0.6212956740055233 0.0009142632500567872 0.040985959316122124 0.06370850682258605
5 0.6012276820838451 0.0007045099865740203 0.03800081479138341 0.038658589720726014
6 0.6071752589195967 0.000626884407378087 0.035044482000942886 0.025749353766441344
7 0.6198538311291486 0.00032723130321496034 0.025092908061783888 0.06996052384376526
8 0.5960314020048827 0.0003776207238249243 0.02649913553533883 0.0430299162864685
9 0.629745373968035 0.0002297023213254668 0.02184328272424895 0.0483766758441925
10 0.6000013051088899 0.0003865656613850995 0.027701951203675107 0.031648169159889224
11 0.6013710389379412 0.0004398936172056

95 0.5877003311179578 2.9917309069676368e-05 0.007910712078727525 0.009924150854349136
96 0.5896382771898061 1.580277736528452e-05 0.005412796268175388 0.014329795837402343
97 0.5899675749242306 2.3339108728188357e-05 0.006782105287601208 0.010100844800472259
98 0.5898962821811438 1.9133886762203324e-05 0.006158200248048223 0.011173269152641297
99 0.5883239989634603 1.797649665326278e-05 0.0057459872755511055 0.008888270854949951
100 0.5943393590860069 4.771269605693565e-06 0.0028565012047003058 0.003054857775568962
101 0.5947473568376154 2.205171451156015e-06 0.002190766234336228 0.0026788240671157836
102 0.5874525960534811 4.117523872082529e-06 0.002822449718569887 0.0058059686422348026
103 0.5953270341269672 6.20525174224672e-06 0.003538736616229189 0.004173449650406838
104 0.5821883110329509 6.732105795927797e-06 0.003598661173520417 0.0030629710853099824
105 0.5785767599008977 2.9386580120778805e-06 0.002294421980864015 0.0021785006672143934
106 0.5780736850574613 7.03900800591346

189 0.5826736618764699 1.010676634954117e-06 0.0013948674512834385 0.003199907839298248
190 0.5815191001165658 1.839899559112085e-06 0.001999649991249216 0.002241207547485828
191 0.5842166789807379 1.5613231741498363e-06 0.0018085112808079555 0.003182631731033325
192 0.5807077561039478 1.320455368158946e-06 0.0016525028504688164 0.0023027750104665755
193 0.5797081049531698 1.2095134634619729e-06 0.0015680138214394963 0.0026128742843866347
194 0.5799731239676476 1.6724240817800782e-06 0.0018758773919323394 0.003289453536272049
195 0.5780091700144112 8.22956960120086e-07 0.0012835333971627828 0.0009962057322263718
196 0.578387132845819 1.4194577657314525e-06 0.0016563919931650163 0.0025533754378557205
197 0.5776646088343114 6.087334730980321e-07 0.0011074349528242802 0.0008558969385921955
198 0.5801614259835333 1.0989591595373394e-06 0.001415146349704471 0.003770083114504814
199 0.5788823559414595 1.5989579634030949e-06 0.0018405145740714567 0.0019640834257006645
200 0.5858663020189852 4

281 0.5785643518902361 6.167434642808373e-08 0.00035959350715937283 0.0004741309396922588
282 0.5823505411390215 6.415035671830547e-08 0.00036399397999048233 0.0005695880949497223
283 0.5792462299577892 1.487068201367019e-07 0.0005534645844379375 0.0011167215369641782
284 0.5806978340260684 1.823494446777742e-07 0.000619520270105066 0.0011462066881358624
285 0.5786978111136705 1.4404582125732713e-07 0.000536231092209446 0.0005678185727447272
286 0.579839315963909 6.173887614298862e-08 0.00035578693423805565 0.0003849600534886122
287 0.5778731349855661 6.799941362823936e-08 0.0003752472932482588 0.0003678829222917557
288 0.5799078200943768 9.457616658080172e-08 0.0004316111743963998 0.0004204456694424152
289 0.578879602951929 6.045823140080914e-08 0.00035248642851566447 0.0003415848361328244
290 0.5920875419396907 6.599802812193697e-08 0.00036642993382852653 0.0005472131259739399
291 0.5986034581437707 1.32757680331934e-07 0.0005311740774275928 0.000470085795968771
292 0.601565603865310

373 0.6258922549895942 2.508249774773132e-08 0.00021651175097915632 0.00021394728682935237
374 0.6199110951274633 1.93685275709e-08 0.0001851676767370824 0.00023531280923634766
375 0.6213054328691214 2.4797448611508922e-08 0.00021776521289399985 0.0002547530131414533
376 0.6228979439474642 2.3502972683699534e-08 0.0002095577082243459 0.00025519330985844134
377 0.6287645560223609 2.5481370628034598e-08 0.0002190048372822589 0.0004082321748137474
378 0.6239245950710028 2.2685415759045852e-08 0.0002024504751095484 0.00022297671530395745
379 0.6218387167900801 2.038664997361157e-08 0.00019023488147248483 0.0002309849439188838
380 0.6235779449343681 1.9130351751696788e-08 0.00018271629393486113 0.00022821519523859025
381 0.6225654729641974 2.543878180214464e-08 0.00021798142585261114 0.00022772747557610275
382 0.6213506660424173 2.3511422160645314e-08 0.0002117732009882557 0.00021321805194020273
383 0.626535678980872 2.0124131096226894e-08 0.0001882938645651628 0.00021473960485309361
384 0.

464 0.6259151948615909 1.3665007148509116e-08 0.00014610002909241053 0.00016564569203183054
465 0.620660524116829 1.3096533091565644e-08 0.0001436231595239249 0.00017090984154492618
466 0.6203635281417519 1.3722612280058312e-08 0.0001496905774070785 0.00016372075304389
467 0.6253919750452042 1.3411408999226232e-08 0.00014443506213740028 0.0001710611954331398
468 0.622279291972518 1.3748455272203659e-08 0.00014608273924938563 0.00016895043198019265
469 0.6254283741582185 1.3198272171311544e-08 0.00014396059230483812 0.00018476028926670552
470 0.6288434260059148 1.350465366744875e-08 0.00014680003426198303 0.00016801631776615977
471 0.6291518229991198 1.3614887222159347e-08 0.0001470497713006776 0.00016960503766313194
472 0.6335766310803592 1.3083163387129124e-08 0.00014300452994889227 0.00016740393824875353
473 0.6322750200051814 1.3140478736874326e-08 0.000142908221078587 0.00016975842881947756
474 0.6224484711419791 1.3142592991224151e-08 0.00014274794206922423 0.00016557254362851382


In [33]:
Param_Analyze = {
    "Alpha": 4.85,
    "InitialSolve": InitialSolve,
    "FinalSolve": 10000,
    "Ndt": 20,
    "TrainingSamples": TrainingSamples,
    "ntrain": ntrain,
    "ntest": ntest,
    "S": S,
    "batch_size": batch_size,
    "learning_rate": learning_rate,
    "epochs": epochs,
    "step_size": step_size,
    "gamma": gamma, 
    "modes": modes,
    "width": width,
    "Initial": 300,
    "Cutoff": 500,
    "Space": Space,
    "FiguresOnly": False
}

In [57]:
%run ./Analyze.ipynb

Inf, L2 = Analyze(Param)

amp_0alpha_4.85 =-=-=-=-=-= 638.32s
Completed
Time for Evaluation:  30.377009974094108
13594


In [185]:
Inf

0.06570043484638344

In [214]:
Inf

0.22213835913611046